In [1]:
import json
import os
import subprocess
import asyncio
import pprint

from bilibili_live_notification import room, config, rate_limit


async def init():
    rate_limit.BILIBILI_API.set(rate_limit.RateLimiter(50, 1))
    room.CACHE_MU.set(asyncio.Lock())

async def get_data(*, event_type, room_id=704808,  revision="master@{now}"):
    data = subprocess.check_output(["git", "show", f"{revision}:event.example.json"])
    event = json.loads(data).get(event_type)
    if event is None:
        return None
    room_data = await room.get_with_cache(event["room_display_id"])
    return dict(
        room=room_data,
        event=event,
    )

async def render(template, data):
    os.environ["temp"] = template
    return config.get("temp", data)

async def render_multiple(template, /, n=3, **kwargs):
    ret = []
    last_data = None
    i = 0
    while len(ret) < n:
        data = await get_data(**kwargs, revision=f"master@{{{i}}}")
        if data is None:
            break
        if data != last_data:
            ret.append(await render(template, data))
            last_data = data
        i+=1
    return ret

## 开播模版

In [14]:
await init()
await render_multiple(
    """\
{% set ri = room.data.room_info %}{% set ai = room.data.anchor_info %}{% set message='[开播]%s：%s\n%s[CQ:image,file=%s]%s' | format(room.name, room.title, room.url, ri.cover or ai.base_info.face or ri.keyframe, MESSAGE) %}{{ dict(message=message, group_id=0) | tojson }}
""",
    event_type="LIVE",
)

['{"group_id": 0, "message": "[\\u5f00\\u64ad]22347045\\uff1aAPEX\\u5927\\u4f1a\\nhttps://live.bilibili.com/22347045[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/028531d9265adde8255ed4581430b5fa4cc76c96.jpg]{{ MESSAGE }}"}',
 '{"group_id": 0, "message": "[\\u5f00\\u64ad]1017\\uff1a\\u4ee5\\u6492\\u7684\\u7ed3\\u5408 \\u5fcf\\u6094 \\u91cc\\u4eba\\u7269\\u89e3\\u9501\\nhttps://live.bilibili.com/1017[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/9ae391c494f0c7d2e8a5e89ffafab590b90a887b.jpg]{{ MESSAGE }}"}',
 '{"group_id": 0, "message": "[\\u5f00\\u64ad]21919321\\uff1a\\u9ad8 \\u6d53 \\u5ea6 \\u4e8c \\u6b21 \\u5143 \\u96c6 \\u4f1a\\nhttps://live.bilibili.com/21919321[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/b6b551cfa6db7bc3d392a50adef02144eeacbc70.jpg]{{ MESSAGE }}"}']

## SuperChat 模版

In [2]:
await init()
await render_multiple(
    """\
{% set e=event.data.data %}{% set user="![face](https://thumb.example.com/20x20/%s)[%s](https://space.bilibili.com/%s)"|format(e.user_info.face | urlencode, e.user_info.uname, e.uid) %}{% set price="![%s](https://img.shields.io/badge/-%s-%s)"|format(e.price, e.price, e.background_bottom_color | urlencode) %}{% set guard_text={1:"总督", 2:"提督", 3:"舰长"}.get(e.user_info.guard_level, e.user_info.guard_level) %}{% set medal='[![%s %s](https://img.shields.io/badge/%s-%s-%s)](https://live.bilibili.com/%s "%s")'|format(e.medal_info.medal_name, e.medal_info.medal_level, e.medal_info.medal_name | urlencode, e.medal_info.medal_level | urlencode, e.medal_info.medal_color | urlencode, e.medal_info.anchor_roomid, e.medal_info.anchor_uname) if e.medal_info else '' %}{{ dict(text=[[price, user, guard_text, medal]|select|join(" "), "#### %s" | format(e.message), e.message_jpn, "######_id: %s_" | format(e.id)] | select | join("\n") ) }}
""",
    event_type="SUPER_CHAT_MESSAGE_JPN",
)

['{\'text\': \'![30](https://img.shields.io/badge/-30-%232A60B2) ![face](https://thumb.example.com/20x20/http%3A//i0.hdslb.com/bfs/face/member/noface.jpg)[hakuna2](https://space.bilibili.com/10183614) 舰长 [![天妇罗 24](https://img.shields.io/badge/%E5%A4%A9%E5%A6%87%E7%BD%97-24-%231a544b)](https://live.bilibili.com/22347045 "凤玲天天Official")\\n#### 来人呐！把朕放锅里煮了给天天吃！\\n######_id: 1411433_\'}',
 '{\'text\': \'![30](https://img.shields.io/badge/-30-%232A60B2) ![face](https://thumb.example.com/20x20/http%3A//i0.hdslb.com/bfs/face/3e7f6bf6b1970c2a9ee597be738a77cdf61562d3.jpg)[杜野04](https://space.bilibili.com/6420216) 舰长 [![IMAS 15](https://img.shields.io/badge/IMAS-15-%23be6686)](https://live.bilibili.com/11101 "RabbitC")\\n#### 羡慕音游打的好的，我自从打了屁股肉，就再也不会打正常下落式音游了，总想判定线会动\\n######_id: 1408909_\'}',
 "{'text': '![30](https://img.shields.io/badge/-30-%232A60B2) ![face](https://thumb.example.com/20x20/http%3A//i2.hdslb.com/bfs/face/4add3acfc930fcd07d06ea5e10a3a377314141c2.jpg)[cici233333](https://space.

## 排行结算模版

In [3]:
await init()
await render_multiple(
    """\
{% set e = event.data.data %}{% set ri = room.data.room_info %}{% set popularity_text='{0:,}'.format(room.popularity)|replace(",", " ") %}{{ dict(group_id=0, message=['[%s No.%s]%s：%s'|format(e.area_name, e.rank, room.name, room.title), room.url, '人气：%s'|format(popularity_text), '状态：%s'|format({0:'未开播',1:'开播于 %s'|format(datetime.fromtimestamp(ri.live_start_time)),2:'轮播中'}.get(ri.live_status, ri.live_status)), '[CQ:image,file=%s]'|format(ri.keyframe) if ri.live_status == 1]|select|join('\n')) }}
""",
    event_type="HOT_RANK_SETTLEMENT",
)

["{'group_id': 0, 'message': '[电台 No.9]22347045：[B限]中文老歌\\nhttps://live.bilibili.com/22347045\\n人气：51 248\\n状态：开播于 2021-03-12 20:35:16\\n[CQ:image,file=http://i0.hdslb.com/bfs/live/keyframe03122220000022347045a9l7qs.jpg]'}",
 "{'group_id': 0, 'message': '[单机游戏 No.6]1017：畅游武侠！ 天地劫\\nhttps://live.bilibili.com/1017\\n人气：2 657 337\\n状态：开播于 2021-03-12 19:11:12\\n[CQ:image,file=http://i0.hdslb.com/bfs/live/keyframe03122225000000001017xavbsp.jpg]'}",
 "{'group_id': 0, 'message': '[虚拟主播 No.6]21320551：【b限】点歌歌回【乙女音】\\nhttps://live.bilibili.com/21320551\\n人气：230 727\\n状态：开播于 2021-03-12 19:58:21\\n[CQ:image,file=http://i0.hdslb.com/bfs/live/keyframe03122226000021320551ikukjv.jpg]'}"]

## 弹幕模版

In [4]:
await init()
for i in await render_multiple(
    """\
{% set i = event.data.info %}{% set msg = i.1 %}{% set user = "%s(%s)"|format(i.2.1,i.2.0) %}{% set medal = "%s|%s(%s)" | format(i.3.1, i.3.0, i.3.3) if i.3 else '' %}{% set guard = {0: "", 1:"总督", 2:"提督", 3:"舰长"}.get(i.7) %}{% set message=[msg, "　　——%s"|format([user,guard,medal]|select|join(' '))]|join('\n') %}{{ message }}
""",
    event_type="DANMU_MSG",
    n=5,
):
    print(i)
    print("\n")

老板大可不必
　　——咕咕的白老鸽(6034957) 天妇罗|5(22347045)


おおおおおおお！！！！
　　——宵夜_sakuya(555951) 舰长 夏厨|24(1321846)


显微镜看散老师手心
　　——安处于世(40706019) 刺儿|11(1017)


hiiro hit me pls
　　——Straybarry(22498076) 舰长 王牛奶|23(21919321)


晚上好
　　——夜庭りち_richi(693383565) 舰长 王牛奶|24(21919321)




## 房间变更

In [6]:
await init()
for i in await render_multiple(
    """\
{% set ri = room.data.room_info %}{% set ai = room.data.anchor_info %}{% set e = event.data.data %}{% set message='[直播间变更]%s：%s\n%s[CQ:image,file=%s]%s' |
format(room.name, e.title, room.url, ri.cover or ai.base_info.face or ri.keyframe, MESSAGE) %}{{ message }}
""",
    event_type="ROOM_CHANGE",
    n=5,
):
    print(i)
    print("\n")

[直播间变更]22571958：【b限】绘本
https://live.bilibili.com/22571958[CQ:image,file=http://i0.hdslb.com/bfs/live/user_cover/fe19c3e1231f2ae25875b47b498ee4132afc1c72.jpg]{{ MESSAGE }}


[直播间变更]21672022：【新衣服】25日 20点！
https://live.bilibili.com/21672022[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/8f30d45e5885dbe919cae75beda1e1676be3b76d.jpg]{{ MESSAGE }}


[直播间变更]1017：可爱像素新游 坎公骑冠剑
https://live.bilibili.com/1017[CQ:image,file=http://i0.hdslb.com/bfs/live/9ae391c494f0c7d2e8a5e89ffafab590b90a887b.jpg]{{ MESSAGE }}


[直播间变更]21320551：【B限】中文歌学唱［大海］
https://live.bilibili.com/21320551[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/d38668bb4775005775ddb81768d4878658a61d2d.jpg]{{ MESSAGE }}


[直播间变更]22347045：[B限]30分没破产请奖励我
https://live.bilibili.com/22347045[CQ:image,file=http://i0.hdslb.com/bfs/live/new_room_cover/cd849deca7dd766b10b041060b4efd10b90d2110.jpg]{{ MESSAGE }}


